### Data Preprocessing
You can use your own way of preprocessing to enhance results. Best results will lead to bonus points.

In [15]:
import pandas as pd
import nltk
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np

In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 89% 23.0M/25.7M [00:00<00:00, 27.2MB/s]
100% 25.7M/25.7M [00:00<00:00, 27.5MB/s]


In [3]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [16]:
reviews = pd.read_csv("/content/IMDB Dataset.csv")

In [17]:
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
reviews.sentiment = reviews.sentiment.map({'positive':1,'negative':0})

In [19]:
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [20]:
reviews.isna().sum()

,0
review,0
sentiment,0


In [21]:
reviews.duplicated().sum()

418

In [22]:
reviews.drop_duplicates(inplace=True)

In [23]:
reviews.reset_index(drop=True, inplace=True)

In [24]:
reviews.duplicated().sum()

0

In [25]:
reviews.shape

(49582, 2)

In [26]:
reviews["length"] = reviews.review.str.len()

In [27]:
reviews.head()

,review,sentiment,length
0,One of the other reviewers has mentioned that ...,1,1761
1,A wonderful little production. <br /><br />The...,1,998
2,I thought this was a wonderful way to spend ti...,1,926
3,Basically there's a family where a little boy ...,0,748
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,1317


In [28]:
reviews.describe()

,sentiment,length
count,49582.000000,49582.000000
mean,0.501876,1310.568230
std,0.500002,990.762238
min,0.000000,32.000000
25%,0.000000,699.000000
50%,1.000000,971.000000
75%,1.000000,1592.000000
max,1.000000,13704.000000


In [29]:

reviews_list = reviews.review.to_list()

In [30]:
len(reviews_list[0].split())

307

In [31]:
reviews.to_csv("clean.csv", index=False)

In [32]:
for i in range(len(reviews_list)):
    words = reviews_list[i].split()
    if len(words) > 200:
        reviews_list[i] = ' '.join(words[:200])

Crearing Tokenizer

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [34]:
tokenizer = Tokenizer()

In [35]:
len(reviews_list)

49582

In [36]:
tokenizer.fit_on_texts(reviews_list)

In [37]:
len(tokenizer.word_index)

103342

In [38]:
X = tokenizer.texts_to_sequences(reviews_list)

In [39]:
Y = reviews.sentiment.to_list()

In [40]:
len(X), len(Y)

(49582, 49582)

In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
X = pad_sequences(X, maxlen=200, padding='pre')

In [43]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size=0.2, random_state=1)

In [44]:
len(val_X), len(val_y), len(train_X), len(train_y)

(9917, 9917, 39665, 39665)

In [45]:
for i in train_X:
    if len(i) > 200:
        print(len(i))

DataSet Class

In [46]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x_data, y_data, transform=None):
        self.reviews = x_data
        self.labels = y_data
        self.transform = transform

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
      try:
        sample = {'review': self.reviews[idx], 'label': self.labels[idx]}
      except:
        sample = {'review': np.zeros_like(self.reviews[0]), 'label': 0}

      if self.transform:
          sample = self.transform(sample)

      return sample

### Sequence models
Following is an example code for simple LSTMs containing one layer. Your implementation should be generic in which user can be able to create multiple layers if required.

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm

class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim):
        super(LSTM, self).__init__()
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim

        self.W_x = nn.Linear(input_dim, 4 * hidden_dim, bias=True)
        self.W_h = nn.Linear(hidden_dim, 4 * hidden_dim, bias=True)

        self.Sigmoid = nn.Sigmoid()
        self.Tanh = nn.Tanh()

    def lstm_step(self, inp, prev_hidden_cell):
        h_prev, c_prev = prev_hidden_cell

        activation = self.W_x(inp) + self.W_h(h_prev)

        ai, af, ac, ao = activation.chunk(4, 1)

        in_gate = self.Sigmoid(ai)
        forget_gate = self.Sigmoid(af)
        cell_gate = self.Tanh(ac)
        out_gate = self.Sigmoid(ao)
        updated_c = forget_gate * c_prev + in_gate * cell_gate
        updated_h = out_gate * self.Tanh(updated_c)

        return updated_h, updated_c

    def forward(self, inp, h, c):
        batch_size, seq_len, embedding_dim = inp.shape
        device = inp.device
        for t in range(seq_len):
            x_t = inp[:, t]
            h, c = self.lstm_step(x_t, (h, c))
        return h, c

In [72]:
class GRU(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super(GRU, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.W_x = nn.Linear(input_dim, 3 * hidden_dim, bias=True)
    self.W_h = nn.Linear(hidden_dim, 3 * hidden_dim, bias=True)

    self.Sigmoid = nn.Sigmoid()
    self.Tanh = nn.Tanh()

  def gru_step(self, inp, prev_hidden):

    activation = self.W_x(inp) + self.W_h(prev_hidden)
    r, z, n = activation.chunk(3, 1)

    reset_gate = self.Sigmoid(r)
    update_gate = self.Sigmoid(z)
    new_gate = self.Tanh(n)

    updated_h = (1 - update_gate) * prev_hidden + update_gate * new_gate
    return updated_h

  def forward(self, inp, h):
    batch_size, seq_len, embedding_dim = inp.shape

    device = inp.device
    for t in range(seq_len):
      x_t = inp[:, t]
      h = self.gru_step(x_t, h)

    return h

In [49]:
class StackedLSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, num_layers, vocab_size):
        super(StackedLSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.drop_out = nn.Dropout(0.25)

        self.embedding = nn.Embedding(vocab_size, input_dim)

        self.lstm_layers = nn.ModuleList([
            LSTM(input_dim if _ == 0 else hidden_dim, hidden_dim) for _ in range(num_layers)
        ])

        self.output_layer = nn.Linear(hidden_dim, 1)

    def forward(self, inp):
        batch_size, seq_len = inp.shape

        inp = self.embedding(inp)
        inp = self.drop_out(inp)

        device = inp.device

        h = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        c = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)

        for layer_index, layer in enumerate(self.lstm_layers):
            h[layer_index], c[layer_index] = layer(inp, h[layer_index].clone(), c[layer_index].clone())
            inp = h[layer_index].unsqueeze(1).repeat(1, seq_len, 1)

        out = self.output_layer(h[-1])
        return out


In [76]:
class StackedGRU(nn.Module):

  def __init__(self, input_dim, hidden_dim, num_layers, vocab_size):
    super(StackedGRU, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers

    self.drop_out = nn.Dropout(0.25)

    self.embedding = nn.Embedding(vocab_size, input_dim)

    self.gru_layers = nn.ModuleList([
        GRU(input_dim if _ == 0 else hidden_dim, hidden_dim) for _ in range(num_layers)
    ])

    self.output_layer = nn.Linear(hidden_dim, 1)

  def forward(self, inp):
    batch_size, seq_len = inp.shape

    inp = self.embedding(inp)
    inp = self.drop_out(inp)

    device = inp.device

    h = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)

    for layer_index, layer in enumerate(self.gru_layers):
      h[layer_index] = layer(inp, h[layer_index].clone())
      inp = h[layer_index].unsqueeze(1).repeat(1, seq_len, 1)

    out = self.output_layer(h[-1])
    return out

# Train

In [51]:
from tqdm import tqdm

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [53]:
device

device(type='cuda')

In [54]:
import torch
import numpy as np
from tqdm import tqdm

def train(model, num_epochs, criterion, optimizer, train_loader, val_loader, device):
    history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': [], 'best_accuracy': -np.inf, 'min_loss': np.inf}

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_corrects = 0, 0

        outer = tqdm(total=len(train_loader.dataset), desc=f'Train Epoch: {epoch + 1} / {num_epochs}')
        for batch in train_loader:
            try:
                inputs = batch['review'].to(device).long()
                labels = batch['label'].view(-1, 1).to(device).float()  # Convert labels to float

                optimizer.zero_grad()
                output = model(inputs)

                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

                # Use sigmoid to convert outputs to probabilities
                output_probs = torch.sigmoid(output)
                predicted = (output_probs > 0.5).float()  # Binary classification
                train_corrects += (predicted == labels).sum().item()

                outer.update(len(inputs))
            except Exception as e:
                print(f"Error: {e}")
                outer.update(len(inputs))

        outer.close()

        train_accuracy = 100. * train_corrects / len(train_loader.dataset)
        train_loss /= len(train_loader)
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Accuracy: {train_accuracy:.2f}%")

        history['train_loss'].append(train_loss)
        history['train_accuracy'].append(train_accuracy)

        # Save the model state
        torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')

        # Validation
        model.eval()
        val_loss, val_corrects = 0, 0

        with torch.no_grad():
            outer = tqdm(total=len(val_loader.dataset), desc='Validating')
            for batch in val_loader:
                inputs = batch['review'].to(device).long()
                labels = batch['label'].view(-1, 1).to(device).float()  # Convert labels to float

                output = model(inputs)
                loss = criterion(output, labels)

                val_loss += loss.item()

                # Use sigmoid to convert outputs to probabilities
                output_probs = torch.sigmoid(output)
                predicted = (output_probs > 0.5).float()  # Binary classification
                val_corrects += (predicted == labels).sum().item()

                outer.update(len(inputs))

            outer.close()

            val_accuracy = 100. * val_corrects / len(val_loader.dataset)
            val_loss /= len(val_loader)
            print(f"Val Loss: {val_loss:.4f}")
            print(f"Val Accuracy: {val_accuracy:.2f}%")

            history['val_loss'].append(val_loss)
            history['val_accuracy'].append(val_accuracy)

            if val_accuracy > history['best_accuracy']:
                history['best_accuracy'] = val_accuracy

            if val_loss < history['min_loss']:
                history['min_loss'] = val_loss

    return history

In [55]:
trainLoader = CustomDataset(train_X, train_y)
valLoader = CustomDataset(val_X, val_y)
# testLoader = CustomDataset(test_x, test_y, transform)

In [56]:
train_loader = torch.utils.data.DataLoader(trainLoader, batch_size=256, shuffle=True, drop_last = True)
val_loader = torch.utils.data.DataLoader(valLoader, batch_size=256, shuffle=True, drop_last= True)
# test_loader = torch.utils.data.DataLoader(testLoader, batch_size=32, shuffle=True)

In [57]:
criterion = nn.BCEWithLogitsLoss()
lstm_model = StackedLSTM(200,256, 3, 103343)

In [58]:
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.current_device())
# print(torch.cuda.get_device_name(torch.cuda.current_device()))


In [59]:
lstm_model.to(device)

StackedLSTM(
  (drop_out): Dropout(p=0.25, inplace=False)
  (embedding): Embedding(103343, 200)
  (lstm_layers): ModuleList(
    (0): LSTM(
      (W_x): Linear(in_features=200, out_features=1024, bias=True)
      (W_h): Linear(in_features=256, out_features=1024, bias=True)
      (Sigmoid): Sigmoid()
      (Tanh): Tanh()
    )
    (1-2): 2 x LSTM(
      (W_x): Linear(in_features=256, out_features=1024, bias=True)
      (W_h): Linear(in_features=256, out_features=1024, bias=True)
      (Sigmoid): Sigmoid()
      (Tanh): Tanh()
    )
  )
  (output_layer): Linear(in_features=256, out_features=1, bias=True)
)

In [60]:
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
lstm_history = train(lstm_model, 10, criterion, optimizer, train_loader, val_loader, device)

Train Epoch: 1 / 10:  99%|█████████▉| 39424/39665 [03:11<00:01, 205.77it/s]


Train Loss: 0.6417
Train Accuracy: 61.41%


Validating:  98%|█████████▊| 9728/9917 [00:05<00:00, 1665.38it/s]


Val Loss: 0.7204
Val Accuracy: 60.57%


Train Epoch: 2 / 10:  99%|█████████▉| 39424/39665 [03:07<00:01, 210.65it/s]


...

Val Loss: 0.4215
Val Accuracy: 84.73%


Train Epoch: 10 / 10:  99%|█████████▉| 39424/39665 [03:06<00:01, 210.91it/s]


Train Loss: 0.1691
Train Accuracy: 93.07%


Validating:  98%|█████████▊| 9728/9917 [00:06<00:00, 1515.29it/s]

Val Loss: 0.4225
Val Accuracy: 85.35%


In [62]:
with open ("lstm_results.json", "w") as f:
    import json
    json.dump(lstm_history, f)

In [77]:
gru_model = StackedGRU(200,256, 7, 103343)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(gru_model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gru_model.to(device)

StackedGRU(
  (drop_out): Dropout(p=0.25, inplace=False)
  (embedding): Embedding(103343, 200)
  (gru_layers): ModuleList(
    (0): GRU(
      (W_x): Linear(in_features=200, out_features=768, bias=True)
      (W_h): Linear(in_features=256, out_features=768, bias=True)
      (Sigmoid): Sigmoid()
      (Tanh): Tanh()
    )
    (1-6): 6 x GRU(
      (W_x): Linear(in_features=256, out_features=768, bias=True)
      (W_h): Linear(in_features=256, out_features=768, bias=True)
      (Sigmoid): Sigmoid()
      (Tanh): Tanh()
    )
  )
  (output_layer): Linear(in_features=256, out_features=1, bias=True)
)

In [78]:
gru_history = train(gru_model, 10, criterion, optimizer, train_loader, val_loader, device)




Train Epoch: 1 / 10:   0%|          | 0/39665 [00:00<?, ?it/s]


Train Epoch: 1 / 10:   1%|          | 256/39665 [00:02<07:35, 86.58it/s]


Train Epoch: 1 / 10:   1%|▏         | 512/39665 [00:05<07:18, 89.19it/s]


Train Epoch: 1 / 10:   2%|▏         | 768/39665 [00:08<07:09, 90.51it/s]


Train Epoch: 1 / 10:   3%|▎         | 1024/39665 [00:11<07:05, 90.82it/s]


Train Epoch: 1 / 10:   3%|▎         | 1280/39665 [00:14<07:42, 83.07it/s]


Train Epoch: 1 / 10:   4%|▍         | 1536/39665 [00:17<07:32, 84.35it/s]


Train Epoch: 1 / 10:   5%|▍         | 1792/39665 [00:20<07:17, 86.60it/s]


Train Epoch: 1 / 10:   5%|▌         | 2048/39665 [00:23<07:06, 88.15it/s]


Train Epoch: 1 / 10:   6%|▌         | 2304/39665 [00:26<06:59, 89.06it/s]


Train Epoch: 1 / 10:   6%|▋         | 2560/39665 [00:29<07:02, 87.76it/s]


Train Epoch: 1 / 10:   7%|▋         | 2816/39665 [00:32<06:56, 88.53it/s]


Train Epoch: 1 / 10:   8%|▊         | 3072/39665 [00:34<06:49, 89.43it/s]


Train Epoch: 1 / 10:   

Train Loss: 0.6879
Train Accuracy: 53.51%


Validating:  98%|█████████▊| 9728/9917 [00:13<00:00, 718.91it/s]


Val Loss: 0.6929
Val Accuracy: 48.18%


Train Epoch: 2 / 10:  99%|█████████▉| 39424/39665 [07:17<00:02, 90.12it/s]


...

Val Loss: 0.3520
Val Accuracy: 83.49%


Train Epoch: 10 / 10:  99%|█████████▉| 39424/39665 [07:18<00:02, 89.95it/s]


Train Loss: 0.2683
Train Accuracy: 88.79%


Validating:  98%|█████████▊| 9728/9917 [00:13<00:00, 717.32it/s]

Val Loss: 0.3662
Val Accuracy: 83.38%


# Testing

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GRU(200,128, 103343)
# model = LSTM(200,256, 103343) #For LSTM
model.load_state_dict(torch.load('/content/model_epoch_5.pth')) # Enter Model's path
model.to(device)

GRU(
  (embedding): Embedding(103343, 200)
  (W_x): Linear(in_features=200, out_features=384, bias=True)
  (W_h): Linear(in_features=128, out_features=384, bias=True)
  (output_layer): Linear(in_features=128, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
  (Tanh): Tanh()
)

In [ ]:
inp = input("Enter review: ")
inp = tokenizer.texts_to_sequences([inp])
inp = pad_sequences(inp, maxlen=200, padding='pre')
inp = torch.tensor(inp).to(device)
output = model(inp)
output_probs = torch.sigmoid(output)
predicted = (output_probs > 0.5).float()
print(output_probs)
if predicted == 1:
    print("Positive")
else:
    print("Negative")

Enter review: Intro was good, but movie was below average, i would say a waste of money
tensor([[0.0101]], device='cuda:0', grad_fn=<SigmoidBackward0>)
Negative
